In [6]:
import os, subprocess, json

target_path = "/home/wii/py_sandbox/pyllow/"
os.chdir(target_path)

# folder_name = "/home/wii/py_sandbox/pyllow/rpc_cg"
# os.mkdir(folder_name)

In [34]:
folder_name = "/home/wii/py_sandbox/pyllow/rpc_cg/"
total_cg = dict()

for root, dirs, files in os.walk(".", topdown=False):
    for name in files:
        if "rpcpy" in root:
            print(os.path.join(root, name))
            filepath = os.path.join(root, name)
            certain_cg = subprocess.check_output(["pycg", filepath])
            certain_cg = json.loads(certain_cg.decode('utf-8').strip())
            total_cg.update(certain_cg)

./rpcpy/client.py
./rpcpy/exceptions.py
./rpcpy/serializers.py
./rpcpy/application.py
./rpcpy/__init__.py
./rpcpy/__version__.py
./rpcpy/openapi.py


In [35]:
# get dependency relation in the script
def clean_cg(cg_dep, entry_name):
    clean_dep = {}
    # get all function calling dependency
    call_deps = get_call_graph(cg_dep)
    
    # find real name in cg_dep
    while entry_name not in cg_dep:
        entry_name = "..." + entry_name
    
    # get all function from the script
    for caller, callee in call_deps.items():
        # if caller.find(entry_name) != -1:
        clean_dep[caller] = call_deps[caller]
    
    # print(clean_dep)
    return clean_dep

In [37]:
# map function to audit event
def function_event(clean_dep):
    caller_audit = {}
    with open("dep_data/standard_audit.json") as open_file:
        standard_audit = json.load(open_file)
    
    for caller, callees in clean_dep.items():
        caller_audit[caller] = set()
        for callee in callees:
            if callee in standard_audit:
                caller_audit[caller].update(set(standard_audit[callee]))

        caller_audit[caller] = list(caller_audit[caller])
        # TODO: need to handle package
        # print(callee)

    # print(caller_audit)
    return caller_audit

In [36]:
def get_all_callees(caller, call_graph, visited=None):
    if visited is None:
        visited = set()
    visited.add(caller)
    callees = call_graph[caller]
    for callee in callees:
        if callee not in visited:
            visited |= get_all_callees(callee, call_graph, visited)
    return visited

def get_call_graph(call_graph):
    result = {}
    for caller in call_graph:
        result[caller] = get_all_callees(caller, call_graph)
    return result

In [40]:
clean_dep = clean_cg(total_cg, "rpcpy")

In [48]:
script_call_event = function_event(clean_dep)

In [63]:
script_call_event

{'rpcpy.client': [],
 'typing.TypeVar': [],
 'rpcpy.client.ClientMeta.__call__': [],
 '<builtin>.isinstance': [],
 'rpcpy.client.Client.__init__': [],
 '<builtin>.TypeError': [],
 '<builtin>.super': [],
 'rpcpy.client.Client.remote_call': [],
 'rpcpy.client.Client._get_url': [],
 'rpcpy.client.Client._get_content': ['exec',
  'builtins.id',
  'compile',
  'sys._getframe'],
 'inspect.signature': ['exec', 'builtins.id', 'compile', 'sys._getframe'],
 '<builtin>.dict': [],
 'rpcpy.client.AsyncClient.remote_call': [],
 'inspect.iscoroutinefunction': [],
 'inspect.isasyncgenfunction': [],
 'rpcpy.openapi.validate_arguments': [],
 'functools.wraps': [],
 'rpcpy.client.AsyncClient.remote_call.wrapper': ['builtins.id',
  'compile',
  'exec',
  'sys._getframe',
  'builtins.id',
  'compile',
  'exec',
  'sys._getframe',
  'builtins.id',
  'compile',
  'exec',
  'sys._getframe',
  'builtins.id',
  'compile',
  'exec',
  'sys._getframe',
  'builtins.id',
  'compile',
  'exec',
  'sys._getframe'],
 

In [66]:
function_audit = {}
file_name = "rpcpy"

for whole_function, audit_event in script_call_event.items():
    if file_name in whole_function and len(audit_event) > 0:
        func_name = whole_function.split(".")[-1]
        if func_name in function_audit:
            function_audit[func_name].extend(audit_event)
            function_audit[func_name] = list(set(function_audit[func_name]))
        else:
            function_audit[func_name] = audit_event

In [67]:
function_audit

{'_get_content': ['exec', 'builtins.id', 'compile', 'sys._getframe'],
 'wrapper': ['exec', 'builtins.id', 'compile', 'sys._getframe'],
 'decode': ['pickle.find_class'],
 'application': ['sys._getframe'],
 'register': ['builtins.id', 'compile', 'exec', 'sys._getframe'],
 'set_type_model': ['builtins.id', 'compile', 'exec', 'sys._getframe'],
 'get_openapi_docs': ['builtins.id', 'compile', 'exec', 'sys._getframe'],
 'respond_openapi': ['builtins.id', 'compile', 'exec', 'sys._getframe'],
 '__call__': ['exec', 'builtins.id', 'compile', 'sys._getframe']}

In [68]:
hook_condition = ""
for func, evt in function_audit.items():
    hook_condition = hook_condition + hook_script.format(func_name=func, evt=evt)

print(hook_condition)


            if func_name == "_get_content":
                if evt not in ['exec', 'builtins.id', 'compile', 'sys._getframe']:
                    raise RuntimeError('Event not allowed')

            if func_name == "wrapper":
                if evt not in ['exec', 'builtins.id', 'compile', 'sys._getframe']:
                    raise RuntimeError('Event not allowed')

            if func_name == "decode":
                if evt not in ['pickle.find_class']:
                    raise RuntimeError('Event not allowed')

            if func_name == "application":
                if evt not in ['sys._getframe']:
                    raise RuntimeError('Event not allowed')

            if func_name == "register":
                if evt not in ['builtins.id', 'compile', 'exec', 'sys._getframe']:
                    raise RuntimeError('Event not allowed')

            if func_name == "set_type_model":
                if evt not in ['builtins.id', 'compile', 'exec', 'sys._getframe']:
          

In [58]:
hook_script = '''
            if func_name == \"{func_name}\":
                if evt not in {evt}:
                    raise RuntimeError('Event not allowed')
'''